In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pandas as pd
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

#navegador.get('https://investidor10.com.br') # /fiis/hglg11/

link_fii = lambda fii: navegador.get(f'https://investidor10.com.br/fiis/{fii}/') #Parâmetro(fii=<nome do investimento>)
#Exemplo link_fii(fii='hglg11')
click_fii = lambda fii: navegador.find_element('xpath', f'{fii}').click()
#Exemplo click_fii(fii='<xpath do arquivo>')
texto_fii = lambda fii: navegador.find_element('xpath', f'{fii}').text
#Exemplo click_fii(fii='<xpath do arquivo>')
repor_ponto = lambda fii: float(fii.replace(",","."))
# Transformar em float -> Exemplo: repor_ponto(fii: <var str>)
def funcao_espera_elemento(tempo=int, elemento=str) -> bool:
    try:    
        wait = WebDriverWait(navegador, tempo) # Quanto tempo esperar?
        element = wait.until(EC.element_to_be_clickable((By.XPATH, f'{elemento}')))  
        return True
    except:
        return False

In [2]:
colunas = ["FII","Cotação", "Dividend Yeld(%)", "P/vP", "Liquidez diária", "Último rendimento", "Vacância(%)"]
valor = []
lista_fiis = ["hglg11", "bcff11", "alzr11", "cpts11", "xplg11", "hgru11", "xpml11", "knri11"]
for fii in lista_fiis:
    link_fii(fii=fii)
    carregamento = funcao_espera_elemento(10, '//*[@id="cards-ticker"]/div[1]/div[2]/div/span')
    if carregamento == True:
        ### Cotação ###
        cotacao = texto_fii(fii='//*[@id="cards-ticker"]/div[1]/div[2]/div/span').split()
        cotacao = repor_ponto(fii=cotacao[1])
        #print(f'A cotação é {cotacao}')
        ### Dividend Yeld ###
        dy = repor_ponto(fii=texto_fii(fii='//*[@id="cards-ticker"]/div[2]/div[2]/div/span').strip("%"))
        #print(f'O Dividend Yeld é {dy}') # Dividend Yeld
        ### P/VP ###
        p_vp = repor_ponto(fii=texto_fii(fii='//*[@id="cards-ticker"]/div[3]/div[2]/span'))
        #print(f'o P/vP é {p_vp}')
        ### Liquidez diária ###
        liquidez_diaria = texto_fii(fii='//*[@id="cards-ticker"]/div[4]/div[2]/span').split()
        liquidez_diaria = repor_ponto(fii=liquidez_diaria[1])
        #print(f'A liquidez diária é {liquidez_diaria}')
        #print("/n")
        ### último rendimento ###
        ultimo_rendimento = texto_fii(fii='//*[@id="table-indicators"]/div[15]/div[2]/div/span').split()
        ultimo_rendimento = repor_ponto(fii=ultimo_rendimento[1])
        ### Vacância
        try:
            vacancia = repor_ponto(texto_fii(fii='//*[@id="table-indicators"]/div[10]/div[2]/div/span').strip("%"))
        except:
            vacancia = 0
        fii = fii.upper()
        valor.append([fii,cotacao, dy, p_vp, liquidez_diaria, ultimo_rendimento, vacancia])
    else:
        print("estourou o tempo!")
tabela = pd.DataFrame(columns=colunas, data=valor)
display(tabela)

,FII,Cotação,Dividend Yeld(%),P/vP,Liquidez diária,Último rendimento,Vacância(%)
0,HGLG11,160.39,9.16,1.05,10.7,1.10,10.00
1,BCFF11,9.35,8.84,0.97,2.5,0.07,0.00
2,ALZR11,114.44,8.51,1.06,1.6,0.83,0.00
3,CPTS11,8.05,10.73,0.90,7.0,0.06,0.00
4,XPLG11,106.44,8.69,0.95,3.6,0.78,2.30
5,HGRU11,133.37,9.07,1.06,4.4,0.85,0.90
6,XPML11,112.28,8.75,1.01,9.9,0.85,4.10
7,KNRI11,158.99,7.66,0.99,3.9,1.00,3.13
